In [6]:
# for take copy from Colab

import shutil

# Your existing code
import sqlite3

def create_database(db_name):
    conn = sqlite3.connect(db_name)
    cur = conn.cursor()
    query = "CREATE TABLE IF NOT EXISTS savings(user_id INTEGER PRIMARY KEY, balance REAL)"
    cur.execute(query)
    conn.commit()
    print("Database connected and savings table created if it didn't exist already")
    return conn

# Path where you want to save the file Googlecolabs
db_conn = create_database("/content/savings.sqlite")

# Path where you want to save the file locally
destination_path = "D:/savings.sqlite"

# Copy the file from Colab to your local disk
shutil.copy("/content/savings.sqlite", destination_path)

print(f"File copied to {destination_path}")


OperationalError: unable to open database file

In [1]:
import sqlite3

def create_database(db_name):
    conn = sqlite3.connect(db_name)
    cur = conn.cursor()
    query = "CREATE TABLE IF NOT EXISTS savings(user_id INTEGER PRIMARY KEY, balance REAL)"
    cur.execute(query)
    conn.commit()
    print(f"Database '{db_name}' created and savings table initialized if it didn't exist already")
#    conn.close()  # Close the connection after creating the database

# Define the name and path of the database file on your local disk
local_db_path = "D:/savings.sqlite"

# Create the database on your local disk
create_database(local_db_path)

print(f"Database created at: {local_db_path}")


Database 'D:/savings.sqlite' created and savings table initialized if it didn't exist already
Database created at: D:/savings.sqlite


In [4]:
#View the database to check it looks correct

import sqlite3
from tabulate import tabulate

def create_database(db_name):
    conn = sqlite3.connect(db_name)
    cur = conn.cursor()
    query = "CREATE TABLE IF NOT EXISTS savings(user_id INTEGER PRIMARY KEY, balance REAL)"
    cur.execute(query)
    conn.commit()
    print(f"Database '{db_name}' created and savings table initialized if it didn't exist already")
    conn.close()  # Close the connection after creating the database

def get_data(conn):
    cur = conn.cursor()
    query = "SELECT * FROM savings"
    cur.execute(query)
    data = cur.fetchall()
    columns = [description[0] for description in cur.description]
    return data, columns

# Define the path to your local database file
local_db_path = "D:/savings.sqlite"  # Replace this with your actual local path

# Create or connect to the database on your local disk
#create_database(local_db_path)

# Connect to the local database file
conn = sqlite3.connect(local_db_path)

table_data, columns = get_data(conn)
print(tabulate(table_data, headers=columns))

user_id    balance
---------  ---------


In [3]:
# Add a balance for a user with id 1

import sqlite3
from tabulate import tabulate

# Define the path to your local database file
local_db_path = "D:/savings.sqlite"  # Replace this with your actual local path

# Connect to the local database file
conn = sqlite3.connect(local_db_path)
                       
def add_new_user(user_id, conn):
  cur = conn.cursor()
  query = f"INSERT INTO savings(user_id,balance) VALUES({str(user_id)},0)"
  cur.execute(query)
  conn.commit()
  print(f"User {str(user_id)} record added")

# run the add new user function to add user with id 1, then run the next cell to get the data again and print it to show that the user with id 1 has been added
add_new_user(1, conn)

IntegrityError: UNIQUE constraint failed: savings.user_id

In [10]:
# Check that the balance was saved by getting the data, then print the whole database table

import sqlite3
from tabulate import tabulate

# Define the path to your local database file
local_db_path = "D:/savings.sqlite"  # Replace this with your actual local path

# Connect to the local database file
conn = sqlite3.connect(local_db_path)

def get_data(conn):
  cur = conn.cursor()
  query = "SELECT * FROM savings"
  cur.execute(query)
  data = cur.fetchall()
  columns = tuple(map(lambda x: x[0], cur.description))
  return data, columns

table_data, columns = get_data(conn)
print(tabulate(table_data, headers=columns))

  user_id    balance
---------  ---------
        1          0


In [12]:
# Check that a user exists before trying to do anything with their data

import sqlite3
from tabulate import tabulate

# Define the path to your local database file
local_db_path = "D:/savings.sqlite"  # Replace this with your actual local path

# Connect to the local database file
conn = sqlite3.connect(local_db_path)

def does_exist(user_id, conn):
  cur = conn.cursor()
  query = f"SELECT * FROM savings WHERE user_id = {str(user_id)}"
  cur.execute(query)
  data = cur.fetchone()
  return data is not None

# run the does_exist function to check that user id 1 exists in the database
print(does_exist(1, conn))

True


In [14]:
# Display a user's balance

import sqlite3
from tabulate import tabulate

# Define the path to your local database file
local_db_path = "D:/savings.sqlite"  # Replace this with your actual local path

# Connect to the local database file
conn = sqlite3.connect(local_db_path)

def get_user_data(user_id, conn):
  if does_exist(user_id, conn):
    cur = conn.cursor()
    query = f"SELECT balance FROM savings WHERE user_id={str(user_id)}"
    cur.execute(query)
    data = cur.fetchone()
    return data[0]
  else:
    return "User does not exist"

# run the get_user_data function to get the balance data for user id 1
balance = get_user_data(1, conn)
print(balance)
     

0.0


In [22]:
# Add funds

import sqlite3
from tabulate import tabulate

# Define the path to your local database file
local_db_path = "D:/savings.sqlite"  # Replace this with your actual local path

# Connect to the local database file
conn = sqlite3.connect(local_db_path)

def add_funds(user_id, amount, conn):
    if does_exist(user_id, conn):
        cur = conn.cursor()
        query = f"UPDATE savings SET balance=balance+{amount} WHERE user_id={user_id}"
        cur.execute(query)  # Execute the update query
        conn.commit()       # Commit the transaction after the update
        return f"Record for user: {user_id} updated - {amount} added"
    else:
        return "User does not exist"

# run the add_funds function to add 15 to the balance for user id 1, then print the result, then print the table to see that it has been added
result = add_funds(1, 15, conn)
print(result)

# Fetch and display the updated data from the table
table_data, columns = get_data(conn)
print(tabulate(table_data, headers=columns))

Record for user: 1 updated - 15 added
  user_id    balance
---------  ---------
        1         30


In [24]:
# Remove funds

import sqlite3
from tabulate import tabulate

# Define the path to your local database file
local_db_path = "D:/savings.sqlite"  # Replace this with your actual local path

# Connect to the local database file
conn = sqlite3.connect(local_db_path)

def remove_funds(user_id, amount, conn):
  if does_exist(user_id, conn):
    cur = conn.cursor()
    query = f"UPDATE savings SET balance=balance - {str(amount)} WHERE user_id={str(user_id)}"
    cur.execute(query)
    data = cur.fetchall()
    return f"Record for user: {str(user_id)} updated - {str(amount)} removed"
  else:
    return "User does not exist"

# run the remove_funds function to remove 15 from the balance for user_id, print the result and then print the whole table to show that the balance has been removed
result = remove_funds(1, 15, conn)
print(result)
table_data, columns = get_data(conn)
print(tabulate(table_data, headers=columns))

Record for user: 1 updated - 15 removed
  user_id    balance
---------  ---------
        1         15


In [25]:
# Exercise 2 - Have a go

import sqlite3
from tabulate import tabulate

# Define the path to your local database file
local_db_path = "D:/savings.sqlite"  # Replace this with your actual local path

# Connect to the local database file
conn = sqlite3.connect(local_db_path)

# Your existing code for connection and functions (add_new_user, get_data, etc.)

def add_user_set():
    for user_id in range(2, 11):  # Loop from 2 to 10 (inclusive)
        add_new_user(user_id)    # Call the function to add new users

# This is just a placeholder function for demonstration purposes
# Replace this with your actual add_new_user function implementation
def add_new_user(user_id):
    cur = conn.cursor()
    query = f"INSERT INTO savings (user_id, balance) VALUES ({user_id}, 0.0)"
    cur.execute(query)
    conn.commit()
    print(f"User with ID {user_id} added")

# Call the function to add users with IDs from 2 to 10
add_user_set()

OperationalError: database is locked

In [26]:
import sqlite3

# Define the path to your local database file
local_db_path = "D:/savings.sqlite"  # Replace this with your actual local path

# Function to add new user
def add_new_user(user_id):
    new_conn = sqlite3.connect(local_db_path)  # Establish a new connection
    try:
        cur = new_conn.cursor()
        query = f"INSERT INTO savings (user_id, balance) VALUES ({user_id}, 0.0)"
        cur.execute(query)
        new_conn.commit()
        print(f"User with ID {user_id} added")
    except sqlite3.Error as e:
        print(f"Error adding user {user_id}: {e}")
    finally:
        new_conn.close()  # Close the connection

# Function to add a set of users
def add_user_set():
    for user_id in range(2, 11):  # Loop from 2 to 10 (inclusive)
        add_new_user(user_id)    # Call the function to add new users

add_user_set()

User with ID 2 added
User with ID 3 added
User with ID 4 added
User with ID 5 added
User with ID 6 added
User with ID 7 added
User with ID 8 added
User with ID 9 added
User with ID 10 added


In [27]:
# Exercise 3 - write a function to remove a user

import sqlite3

# Define the path to your local database file
local_db_path = "D:/savings.sqlite"  # Replace this with your actual local path

# Function to remove an existing user
def remove_user(user_id):
    conn = sqlite3.connect(local_db_path)
    try:
        cur = conn.cursor()
        query = f"DELETE FROM savings WHERE user_id={user_id}"
        cur.execute(query)
        conn.commit()
        print(f"User with ID {user_id} removed")
    except sqlite3.Error as e:
        print(f"Error removing user {user_id}: {e}")
    finally:
        conn.close()  # Close the connection

# Call the function to remove a user (for example, user_id = 3)
remove_user(3)

User with ID 3 removed


In [29]:
#Exit the app, closing the database connection

import sqlite3

# Define the path to your local database file
local_db_path = "D:/savings.sqlite"  # Replace this with your actual local path

# Function to establish database connection
def connect_to_db(db_path):
    conn = sqlite3.connect(db_path)
    return conn

# Function to close and nullify the database connection
def exit_app(conn):
    if conn:
        conn.close()
        conn = None
        print("Exiting app...")
    return conn

# Establish the database connection
db_conn = connect_to_db(local_db_path)

# Close and nullify the connection
conn = exit_app(conn)


Exiting app...


In [40]:
import sqlite3
from tabulate import tabulate

# Define the path to your local database file
local_db_path = "D:/savings.sqlite"  # Replace this with your actual local path


# Establish a connection to the database
conn = sqlite3.connect(local_db_path)

# Add a new user with ID 5 (example)
#add_new_user(5, conn)

# Fetch data from the 'savings' table and display it
table_data, columns = get_data(conn)
print(tabulate(table_data, headers=columns))

# Close the connection when done
#conn.close()

user_id = 4
# gets a copy of the data from the savings table
# ---------------------------------------------------------------------------------------------------------------------------------------------------
def get_data(conn):
  cur = conn.cursor()
  query = "SELECT * FROM savings"
  cur.execute(query)
  data = cur.fetchall()
  columns = tuple(map(lambda x: x[0], cur.description))
  return data, columns

# prints the balance for.   The table will have a number of rows corresponding to the number of users plus a header row.
# it will be possible to get a list of users from this table
# ---------------------------------------------------------------------------------------------------------------------------------------------------
def get_user_data(user_id, conn):
  if does_exist(user_id, conn):
    cur = conn.cursor()
    query = f"SELECT balance FROM savings WHERE user_id={str(user_id)}"
    cur.execute(query)
    data = cur.fetchone()
    return data[0]
  else:
    return "User does not exist"

# check that a user with a given id does or doesn't exist in the savings table
# returns True or False
# ---------------------------------------------------------------------------------------------------------------------------------------------------
def does_exist(user_id, conn):
  cur = conn.cursor()
  query = f"SELECT * FROM savings WHERE user_id = {str(user_id)}"
  cur.execute(query)
  data = cur.fetchone()
  return data is not None

# adds a user with the given id to the savings table
# returns a message to confirm that this has happened
# ---------------------------------------------------------------------------------------------------------------------------------------------------
def add_new_user(user_id, conn):
  cur = conn.cursor()
  query = f"INSERT INTO savings(user_id,balance) VALUES({str(user_id)},0)"
  cur.execute(query)
  conn.commit()
  print(f"User {str(user_id)} record added")
  return True

# removes a user with the given id from the savings table after checking that the user does exist
# returns a message to confirm that this has happened or that the user doesn't exist
# ----------------------------------------------------------------------------------------------------------------------------------
def remove_user(user_id, conn):
  if does_exist(user_id, conn):
    cur = conn.cursor()
    query = f"DELETE FROM savings WHERE user_id={str(user_id)}"
    cur.execute(query)
    print(f"User {str(user_id)} deleted")
    return True
  else:
    print("User does not exist")
    return False

# updates the balance for the given user by adding the given amount onto the balance
# returns a message to confirm that this has happened or that the user doesn't exist
# ----------------------------------------------------------------------------------------------------------------------------------
def add_funds(user_id, amount, conn):
  if does_exist(user_id, conn):
    cur = conn.cursor()
    query = f"UPDATE savings SET balance=balance+{str(amount)} WHERE user_id={str(user_id)}"
    cur.execute(query)
    data = cur.fetchall()
    print(f"Record for user: {str(user_id)} updated - {str(amount)} added")
    return True
  else:
    print("User does not exist")
    return False

# updates the balance for the given user by subtracting the given amount from the balance
# returns a message to confirm that this has happened or that the user doesn't exist
# ----------------------------------------------------------------------------------------------------------------------------------
def remove_funds(user_id, amount, conn):
  if does_exist(user_id, conn):
    cur = conn.cursor()
    query = f"UPDATE savings SET balance=balance - {str(amount)} WHERE user_id={str(user_id)}"
    cur.execute(query)
    data = cur.fetchall()
    print(f"Record for user: {str(user_id)} updated - {str(amount)} removed")
    return True
  else:
    print("User does not exist")
    return False

# nullifies the connection so that is can't accidentally be used again
# ----------------------------------------------------------------------------------------------------------------------------------
def exit_app(conn):
  conn.close()
  conn = None
  print("Exiting app...")
  return conn



  user_id    balance
---------  ---------
        1         30
        2          0
        4          0
        5        100
        6          0
        7          0
        8          0
        9          0
       10          0


In [16]:
import sqlite3
from tabulate import tabulate


local_db_path = "D:/savings.sqlite"  # Replace this with your actual local path


def add_new_user(user_id, conn):
    if not does_exist(user_id, conn):  # Check if the user exists before insertion
        try:
            cur = conn.cursor()
            query = f"INSERT INTO savings(user_id,balance) VALUES({str(user_id)},0)"
            cur.execute(query)
            conn.commit()
            print(f"User {str(user_id)} record added")
        except sqlite3.Error as e:
            print(f"Error adding user {user_id}: {e}")
    else:
        print(f"User {user_id} already exists")

def add_funds(user_id, amount, conn):
    if does_exist(user_id, conn):
        try:
            with conn:  # Use a with statement to ensure proper closure
                cur = conn.cursor()
                query = f"UPDATE savings SET balance = balance + {str(amount)} WHERE user_id = {str(user_id)}"
                cur.execute(query)
                print(f"Record for user: {str(user_id)} updated - {str(amount)} added")
        except sqlite3.Error as e:
            print(f"Error updating balance for user {user_id}: {e}")
    else:
        print("User does not exist")

# Function to create the database
def create_database(db_name):
    conn = sqlite3.connect(db_name)
    cur = conn.cursor()
    query = "CREATE TABLE IF NOT EXISTS savings(user_id INTEGER PRIMARY KEY, balance REAL)"
    cur.execute(query)
    conn.commit()
    print("Database connected and savings table created if it didn't exist already")
    return conn

# Adjusted function to setup the database with initial data
def setup_database(conn):
    starting_balances = [44, 5, 42, 32, 25, 5, 26, 39, 45, 29]
    for id in range(1, 11):
        add_new_user(id, conn)
        add_funds(id, starting_balances[id - 1], conn)
    print("Test data added")

def does_exist(user_id, conn):
    cur = conn.cursor()
    query = f"SELECT user_id FROM savings WHERE user_id = {user_id}"
    cur.execute(query)
    data = cur.fetchone()
    return data is not None

def get_data(conn):
    cur = conn.cursor()
    query = "SELECT * FROM savings"
    cur.execute(query)
    data = cur.fetchall()
    columns = [description[0] for description in cur.description]
    return data, columns

# Adjusted code to create the database in the D:/ path
local_db_path = "D:/savings.sqlite"  # Updated path
conn = create_database(local_db_path)  # Create the database

# Setup the database with initial data
setup_database(conn)  # Change 'con' to 'conn'

# Fetch and display the data
data, columns = get_data(conn)
print(tabulate(data, headers=columns))

# Close the database connection

def exit_app(conn):
    conn.close()
    print("Exiting the application...")
exit_app(conn) 

Database connected and savings table created if it didn't exist already
User 1 already exists
Record for user: 1 updated - 44 added
User 2 already exists
Record for user: 2 updated - 5 added
User 3 already exists
Record for user: 3 updated - 42 added
User 4 already exists
Record for user: 4 updated - 32 added
User 5 already exists
Record for user: 5 updated - 25 added
User 6 already exists
Record for user: 6 updated - 5 added
User 7 already exists
Record for user: 7 updated - 26 added
User 8 already exists
Record for user: 8 updated - 39 added
User 9 already exists
Record for user: 9 updated - 45 added
User 10 already exists
Record for user: 10 updated - 29 added
Test data added
  user_id    balance
---------  ---------
        1        132
        2         15
        3        126
        4         96
        5         75
        6         15
        7         78
        8        117
        9        135
       10         87
Exiting the application...


In [2]:
import sqlite3
import time
from tabulate import tabulate

local_db_path = "D:/savings.sqlite"  # Replace this with your actual local path

def add_new_user(user_id, conn):
    query = f"INSERT INTO savings(user_id, balance) VALUES({str(user_id)}, 0)"
    try:
        conn.execute(query)
        print(f"User {str(user_id)} record added")
    except sqlite3.IntegrityError as e:
        print(f"Error adding user {user_id}: {e}")

# Define the path to your local database file
local_db_path = "D:/savings.sqlite"  # Replace this with your actual local path

# Connect to the local database file
conn = sqlite3.connect(local_db_path)

# Begin a transaction
conn.execute("BEGIN")

# Add a new user with a different user ID and a balance of 0
new_user_id = 2  # Replace with a different user ID
add_new_user(new_user_id, conn)

# Commit the transaction
conn.commit()

# Allow a short delay before accessing the database again
time.sleep(1)

# Fetch and display the updated data from the 'savings' table
cur = conn.cursor()
cur.execute("SELECT * FROM savings")
data = cur.fetchall()
columns = [description[0] for description in cur.description]

print(tabulate(data, headers=columns))


Error adding user 2: UNIQUE constraint failed: savings.user_id
  user_id    balance
---------  ---------
        1        132
        2         15
        3        126
        4         96
        5         75
        6         15
        7         78
        8        117
        9        135
       10         87
       11          0
